my dataset contains 16 training and 21 testing video clips. 30652 (15328 training, 15324 testing) framesin total. The training
videos contain video with normal situations. The testing videos contain videos with both standard and abnormal events.
now what i want to do that is first extract flames from each video of both folder and them save them in flames folder which
is in both folders and then build a system to automate the task of analyzing video surveillance. We will analyze the video
feed in real-time and identify any abnormal activities like violence or theft.

In [1]:
import glob
import os
import cv2
import imutils
import argparse
import numpy as np
from PIL import Image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, ConvLSTM2D, Conv3DTranspose
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.image import img_to_array, load_img

In [2]:
# Function to store image in array
def store_inarray(image_path, lst):
    image = load_img(image_path)
    image = img_to_array(image)
    image = cv2.resize(image, (227, 227), interpolation=cv2.INTER_AREA)
    gray = 0.2989 * image[:, :, 0] + 0.5870 * image[:, :, 1] + 0.1140 * image[:, :, 2]
    lst.append(gray)

In [3]:
# Paths
train_path = r'D:\python projects\Intelligent_Video_Surveillance\Avenue Dataset\training_videos'
fps = 5
train_images_path = os.path.join(train_path, 'frames')
os.makedirs(train_images_path, exist_ok=True)

In [4]:
train_videos = glob.glob(os.path.join(train_path, '*.avi'))
store_image_train = []

if not os.listdir(train_images_path):
    for video_path in train_videos:
        video_name = os.path.basename(video_path)
        video_folder = os.path.join(train_images_path, video_name[:-4])
        os.makedirs(video_folder, exist_ok=True)

        # Open the video file
        cap = cv2.VideoCapture(video_path)

        frame_count = 0

        # Read frames until the video is complete
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Save frame as an image file
            frame_path = os.path.join(video_folder, '{:03d}.jpg'.format(frame_count))
            cv2.imwrite(frame_path, frame)

            store_inarray(frame_path, store_image_train)

            frame_count += 1

        cap.release()
else:
    print("flames are already avalable")
# Print the number of training frames extracted
print("Training frames extracted:", len(store_image_train))


flames are already avalable
Training frames extracted: 0


In [6]:
if len(store_image_train) > 0:
    store_image_train = np.array(store_image_train)
    a, b, c = store_image_train.shape

    store_image_train.resize(b, c, a)
    store_image_train = (store_image_train - store_image_train.mean()) / store_image_train.std()
    store_image_train = np.clip(store_image_train, 0, 1)
    np.save('training.npy', store_image_train)
    print("Training images saved as numpy array.")
else:
    print("No training images found.")


No training images found.
No testing images found.


In [7]:
# Load the training and testing datasets
training_data = np.load('./Avenue Dataset/training_videos/frames/training.npy')

In [8]:
print("Training data shape:", training_data.shape)

Training data shape: (227, 227, 15328)
Testing data shape: (227, 227, 15324)


In [9]:
frames_train = training_data.shape[2]
frames_train = frames_train - frames_train % 10
training_data = training_data[:, :, :frames_train]
training_data = training_data.reshape(-1, 227, 227, 10)
training_data = np.expand_dims(training_data, axis=4)

In [10]:
stae_model=Sequential()

stae_model.add(Conv3D(filters=128,kernel_size=(11,11,1),strides=(4,4,1),padding='valid',input_shape=(227,227,10,1),activation='tanh'))
stae_model.add(Conv3D(filters=64,kernel_size=(5,5,1),strides=(2,2,1),padding='valid',activation='tanh'))
stae_model.add(ConvLSTM2D(filters=64,kernel_size=(3,3),strides=1,padding='same',dropout=0.4,recurrent_dropout=0.3,return_sequences=True))
stae_model.add(ConvLSTM2D(filters=32,kernel_size=(3,3),strides=1,padding='same',dropout=0.3,return_sequences=True))
stae_model.add(ConvLSTM2D(filters=64,kernel_size=(3,3),strides=1,return_sequences=True, padding='same',dropout=0.5))
stae_model.add(Conv3DTranspose(filters=128,kernel_size=(5,5,1),strides=(2,2,1),padding='valid',activation='tanh'))
stae_model.add(Conv3DTranspose(filters=1,kernel_size=(11,11,1),strides=(4,4,1),padding='valid',activation='tanh'))

stae_model.compile(optimizer='adam',loss='mean_squared_error',metrics=['accuracy'])

In [11]:
epochs=5
batch_size=1
callback_save = ModelCheckpoint("saved_model.h5", monitor="mean_squared_error", save_best_only=True)
callback_early_stopping = EarlyStopping(monitor='val_loss', patience=3)

In [12]:
target_data=training_data.copy()
stae_model.fit(training_data, target_data, batch_size=batch_size, epochs=epochs, callbacks=[callback_save, callback_early_stopping])

Epoch 1/5
1532/1532 [==============================] - 1799s 1s/step - loss: 0.0841 - accuracy: 0.7182
Epoch 2/5
1532/1532 [==============================] - 1995s 1s/step - loss: 0.0692 - accuracy: 0.7372
Epoch 3/5
1532/1532 [==============================] - 2157s 1s/step - loss: 0.0685 - accuracy: 0.7379
Epoch 4/5
1532/1532 [==============================] - 2287s 1s/step - loss: 0.0454 - accuracy: 0.7570
Epoch 5/5
1532/1532 [==============================] - 2757s 2s/step - loss: 0.0304 - accuracy: 0.7736


In [13]:
stae_model.save("saved_model.h5")

In [14]:
stae_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 55, 55, 10, 128)   15616     
                                                                 
 conv3d_1 (Conv3D)           (None, 26, 26, 10, 64)    204864    
                                                                 
 conv_lstm2d (ConvLSTM2D)    (None, 26, 26, 10, 64)    295168    
                                                                 
 conv_lstm2d_1 (ConvLSTM2D)  (None, 26, 26, 10, 32)    110720    
                                                                 
 conv_lstm2d_2 (ConvLSTM2D)  (None, 26, 26, 10, 64)    221440    
                                                                 
 conv3d_transpose (Conv3DTra  (None, 55, 55, 10, 128)  204928    
 nspose)                                                         
                                                        

In [2]:
import cv2
import numpy as np
from keras.models import load_model
import imutils

def mean_squared_loss(x1, x2):
    difference = x1 - x2
    a, b, c, d, e = difference.shape
    n_samples = a * b * c * d * e
    sq_difference = difference**2
    Sum = sq_difference.sum()
    distance = np.sqrt(Sum)
    mean_distance = distance / n_samples
    return mean_distance

model = load_model("saved_model.h5")

path = './test/4.mp4'
cap = cv2.VideoCapture(path)
print(cap.isOpened())

while cap.isOpened():
    imagedump = []
    ret, frame = cap.read()

    if frame is None:
        break

    for i in range(10):
        ret, frame = cap.read()
        if frame is None:
            break

        image = imutils.resize(frame, width=700, height=600)
        frame = cv2.resize(frame, (227, 227), interpolation=cv2.INTER_AREA)
        gray = 0.2989 * frame[:, :, 0] + 0.5870 * frame[:, :, 1] + 0.1140 * frame[:, :, 2]
        gray = (gray - gray.mean()) / gray.std()
        gray = np.clip(gray, 0, 1)
        imagedump.append(gray)

    if frame is None:
        break

    imagedump = np.array(imagedump)
    imagedump.resize(227, 227, 10)
    imagedump = np.expand_dims(imagedump, axis=0)
    imagedump = np.expand_dims(imagedump, axis=4)

    output = model.predict(imagedump)

    loss = mean_squared_loss(imagedump, output)
    print(loss)
    if frame.any() is None:
        print("none")

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

    if loss > 0.000735:
        print('Abnormal Event Detected')
        cv2.putText(image, "Abnormal Event", (100, 80), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 4)

    cv2.imshow("video", image)

cap.release()
cv2.destroyAllWindows()

True
1/1 [==============================] - 1s 654ms/step
0.0007004712294400224
1/1 [==============================] - 0s 236ms/step
0.000700438967061304
1/1 [==============================] - 0s 203ms/step
0.0007004682630005211
1/1 [==============================] - 0s 225ms/step
0.0006999945910800179
1/1 [==============================] - 0s 204ms/step
0.0007117500412866372
1/1 [==============================] - 0s 191ms/step
0.0007139195078812982
1/1 [==============================] - 0s 222ms/step
0.0007321339073991451
1/1 [==============================] - 0s 233ms/step
0.0007494207320123749
Abnormal Event Detected
1/1 [==============================] - 0s 199ms/step